# Arthur Sandbox Example: OpenAI question-answering

In this guide, we'll use a question-answering dataset from Huggingface and the GPT3 endpoint from OpenAI to onboard a new streaming model to the Arthur platform. Then we will use Arthur to analyze our model.


# Outline

Read on for an overview of everything this notebook will cover. **[Click here to dive straight into the code.](#Imports)**

## Onboarding

Onboarding is the process of setting up your model to be monitored by Arthur. You specify the type of data your model ingests, send a reference dataset to provide a baseline of the distribution of your data, and you configure additional settings among the services Arthur offers.

**Arthur does not need your model object itself to monitor performance - only predictions are required**

All you need to monitor your model with Arthur is to upload the predictions your model makes: Arthur computes analytics about your model based on that prediction data. This data can be computed directly by your model in a script or notebook like this one to be uploaded to the platform, or can be fetched from an external database to be sent to Arthur.

### Getting Model Predictions
We'll prepare a sample from a question-answering dataset and generate answers from the GPT-3 endpoint.

### Registering Model with Arthur
We'll configure our model's attributes and save the model to the Arthur platform.

### Sending Inferences
We'll send model inferences (inputs and predictions) to the Arthur platform.


## Model Monitoring and Analysis

Once onboarding is complete and you have inferences uploaded to the platform, you can use Arthur to get model monitoring insights.

We will evaluate the Arthur-computed metrics to identify trends in our model's inferences

---

# Setup & Imports

In [55]:
# ensure required packages are installed
#  don't worry, our requirements are flexible!

! pip install -r requirements.txt > /dev/null

In [56]:
from datetime import datetime, timedelta
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import shortuuid

---

# Onboarding

## Loading the Dataset

We're using the [SciQ dataset from Huggingface](https://huggingface.co/datasets/sciq)

The SciQ dataset contains 13,679 crowdsourced science exam questions about Physics, Chemistry and Biology, among others. The questions are in multiple-choice format with 4 answer options each. For the majority of the questions, an additional paragraph with supporting evidence for the correct answer is provided.

In [57]:
from datasets import load_dataset

In [58]:
sciq_dataset = load_dataset("sciq")

Found cached dataset sciq (/Users/maxcembalest/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493)


  0%|          | 0/3 [00:00<?, ?it/s]

In [59]:
sciq_df = sciq_dataset.data['train'].table.to_pandas()
sciq_df_inf = sciq_dataset.data['test'].table.to_pandas()

In [11]:
sciq_df_sample = sciq_df.sample(50)
sciq_df_inf_sample = sciq_df_inf.sample(50)

## Create the full LLM inputs by concatenating the actual question after a sentence/paragraph of supporting information

In [20]:
gpt3_inputs = [
    f"What is the answer to this question? question: {row.question}, answer: " 
    for _, row in sciq_df_sample.iterrows()
]

gpt3_inputs_inf = [
    f"What is the answer to this question? question: {row.question}, answer: " 
    for _, row in sciq_df_inf_sample.iterrows()
]

In [13]:
gpt3_inputs

['What is the answer to this question? question: What type of compound contains atoms of two or more different elements in its ring structure?, answer: ',
 'What is the answer to this question? question: The most common curved mirrors are what?, answer: ',
 'What is the answer to this question? question: Global warming will raise ocean levels due to melt water from glaciers and the greater volume of what?, answer: ',
 'What is the answer to this question? question: The heme parts of a hemoglobin molecule bind with what element?, answer: ',
 'What is the answer to this question? question: Because atoms are always electrically neutral, for each added proton, one of what is also added?, answer: ',
 'What is the answer to this question? question: What type of muscle is found in the walls of other internal organs such as the stomach?, answer: ',
 'What is the answer to this question? question: When does the diploid zygote form?, answer: ',
 'What is the answer to this question? question: Wh

# Get OpenAI responses

Using the endpoint for the `text-davinci-003` model, we get an answer to each question from our sample dataframe

In [14]:
import openai

In [16]:
from typing import List

def get_gpt3_responses(inputs: List[str]):
    """
    Applies GPT3 to the inputs
    """
    
    responses = []
    
    for inp in inputs:
        openai_response = openai.Completion.create(
            model="text-davinci-003", 
            prompt=inp, 
            temperature=0.1, 
            logprobs=1)
        gpt3_answer = openai_response["choices"][0]["text"]
        responses.append(openai_response)
    
    return responses

In [17]:
responses = get_gpt3_responses(gpt3_inputs)

In [21]:
responses_inf = get_gpt3_responses(gpt3_inputs_inf)

In [24]:
gpt3_response_texts = [c['choices'][0]['text'] for c in responses]
gpt3_response_texts_inf = [c['choices'][0]['text'] for c in responses_inf]

#### Get token likelihoods

In [22]:
gpt3_tokens_ref = [resp['choices'][0]['logprobs']['tokens'] for resp in responses]
gpt3_probabilities_ref = [list(np.exp(resp['choices'][0]['logprobs']['token_logprobs'])) for resp in responses]
gpt3_token_likelihoods_ref = [[
    {t: p}
    for t, p in zip(gpt3_tokens_ref[i], gpt3_probabilities_ref[i])] for i in range(len(gpt3_inputs))]

gpt3_tokens_inf = [resp['choices'][0]['logprobs']['tokens'] for resp in responses_inf]
gpt3_probabilities_inf = [list(np.exp(resp['choices'][0]['logprobs']['token_logprobs'])) for resp in responses_inf]
gpt3_token_likelihoods_inf = [[
    {t: p}
    for t, p in zip(gpt3_tokens_inf[i], gpt3_probabilities_inf[i])] for i in range(len(gpt3_inputs_inf))]

#### Get finish reasons

In [25]:
gpt3_response_finish_reasons_ref = [c['choices'][0]['finish_reason'] for c in responses]
gpt3_response_finish_reasons_inf = [c['choices'][0]['finish_reason'] for c in responses_inf]

#### Get readability scores

In [34]:
import textstat
gpt3_readability_ref = [textstat.flesch_reading_ease(o) for o in gpt3_response_texts]
gpt3_readability_inf = [textstat.flesch_reading_ease(o) for o in gpt3_response_texts_inf]

#### Get correctness of answers

In [28]:
def get_correctness(outputs, answers):
    correctnesses = []
    for a, o in zip(answers, outputs):
        in_ = True
        for word in a.replace(',','').split():
            if word.lower() not in o.lower():
                in_=False
        correctnesses.append(in_)
    return correctnesses

In [29]:
gpt3_correctness_ref = get_correctness(
    gpt3_response_texts,
    list(sciq_df_sample.correct_answer.values)
)

gpt3_correctness_inf = get_correctness(
    gpt3_response_texts_inf,
    list(sciq_df_inf_sample.correct_answer.values)
)

#### Use ChatGPT to get LLM-generated feedback

In [30]:
def get_pseudofeedback(inputs: List[str]):
    responses = []
    for inp in inputs:
        print(inp)
        messages = [
            {"role": "system", "content": "You give short written feedback saying if an answer made sense for a given question. You also rate the answer out of 10 (0=worst,10=best)"},
            {"role": "user", "content": "What is the answer to this question? question: What makes breathing difficult due to respiratory system disease?, answer: The answer to this question is the answer to your question.\n\nIt is important to know the answer to this question because"},
            {"role": "assistant", "content": "You didn't actually answer the question, thats bad and confusing. 1"},
            {"role": "user", "content": "What is the answer to this question? question: What makes breathing difficult due to respiratory system disease?, answer: Respiratory system diseases can make breathing difficult by causing inflammation, damage, or blockages in the airways or lung tissues. These can include conditions such as asthma, chronic obstructive pulmonary disease (COPD), pneumonia, lung cancer, and others. The inflammation and swelling can narrow the airways, making it harder for air to flow in and out of the lungs. Damage to the lung tissues can reduce their ability to expand and contract, which can also make breathing difficult. Additionally, excess mucus or fluid in the lungs can further narrow the airways and make it hard to breathe. These factors can cause symptoms such as shortness of breath, coughing, wheezing, and fatigue."},
            {"role": "assistant", "content": "Thank you very much, that makes a lot of sense! 10"},
            {"role": "user", "content": inp}
        ]

        openai_response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=messages
        )
        chat_gpt_answer = openai_response["choices"][0]["message"]["content"]
        print(chat_gpt_answer, "\n=====\n")
        responses.append(openai_response)
    
    return responses

In [31]:
gpt3_pseudofeedback_ref = get_pseudofeedback([inp + rt for inp, rt in zip(gpt3_inputs, gpt3_response_texts)])
gpt3_pseudofeedback_inf = get_pseudofeedback([inp + rt for inp, rt in zip(gpt3_inputs_inf, gpt3_response_texts_inf)])



What is the answer to this question? question: What type of compound contains atoms of two or more different elements in its ring structure?, answer: 

A heterocyclic compound.
Thank you for your answer. It seems to make sense. 8 
=====

What is the answer to this question? question: The most common curved mirrors are what?, answer: 
Concave mirrors.
Your answer is correct and concise. Good job! 8 
=====

What is the answer to this question? question: Global warming will raise ocean levels due to melt water from glaciers and the greater volume of what?, answer: 

the greater volume of water in the oceans from thermal expansion.
Thank you for your answer! It is concise and accurate. 8 
=====

What is the answer to this question? question: The heme parts of a hemoglobin molecule bind with what element?, answer:  Oxygen
Your answer is correct and makes sense. 10 
=====

What is the answer to this question? question: Because atoms are always electrically neutral, for each added proton, one

Yes, that is correct. Your answer makes sense. 10 
=====

What is the answer to this question? question: By how many weeks do all major organs start developing?, answer: 

By 8 weeks, all major organs start developing.
Great answer! That is correct and clear. 10 
=====

What is the answer to this question? question: All chordates are deuterostomes possessing a what?, answer: 

Chordates are deuterostomes possessing a notochord.
Your answer is correct and clear. 10 
=====

What is the answer to this question? question: Fertilization occurs if a sperm enters the egg while it is passing through what tube?, answer:  Fallopian tube
Your answer is correct and makes sense in the given context. Well done! 10 
=====

What is the answer to this question? question: While normal light microscopes can magnify objects up to about 1,500 times, what advanced instruments can magnify objects up to 1,000,000 times?, answer: 

Electron microscopes can magnify objects up to 1,000,
This answer is incomplete

The answer provided is correct and makes sense. 8 
=====

What is the answer to this question? question: How do many mammals control their body temperature?, answer: 

Many mammals control their body temperature by regulating their behavior, such as seeking
Sorry, your answer seems incomplete as you didn't complete your sentence. Please check your response again and provide a complete answer. 
=====

What is the answer to this question? question: Traits may also be controlled by multiple alleles or multiple what?, answer: 

Multiple genes.
Your answer is partially correct. It is "multiple alleles" which can control traits, but it is also true that multiple genes can control traits. So the complete answer would be "Traits may also be controlled by multiple alleles or multiple genes." I would rate this answer a 7 out of 10 for partially answering the question. 
=====

What is the answer to this question? question: What do you call health-promoting molecules that inhibit the oxidation of 

Yes, that answer is correct! 10 
=====

What is the answer to this question? question: What are the two terms that designate how well aqueous solutions conduct electricity?, answer:  Electrolytic conductivity and specific conductivity.
Your answer is correct and concise. Well done! 8 
=====

What is the answer to this question? question: Which membranous organelles in a cell sequence enzyme and electron molecules for efficient cellular respiration?, answer:  Mitochondria
Your answer is correct and makes sense. 10 
=====

What is the answer to this question? question: What determines how long a planet's orbit is around the sun?, answer: 

The length of a planet's orbit around the sun is determined by its
distance from the sun and its gravitational pull. This is known as Kepler's Third Law of Planetary Motion, which states that the square of a planet's orbital period is proportional to the cube of its average distance from the sun. In other words, the farther a planet is from the sun, th

In [32]:
def get_feedback_scores(feedback_strings):
    feedback_ints = []
    for fs in feedback_strings:
        score = -1
        fs = fs.replace('out of 10', '')
        for num_string in np.arange(11).astype(str):
            if num_string in fs:
                score = int(num_string)
        feedback_ints.append(score)
    return feedback_ints

In [33]:
gpt3_pseudofeedback_messages_ref = [c['choices'][0]['message']['content'] for c in gpt3_pseudofeedback_ref]
gpt3_pseudofeedback_messages_inf = [c['choices'][0]['message']['content'] for c in gpt3_pseudofeedback_inf]

gpt3_feedback_scores_ref = get_feedback_scores(gpt3_pseudofeedback_messages_ref)
gpt3_feedback_scores_inf = get_feedback_scores(gpt3_pseudofeedback_messages_inf)

### Create inference dataframes

In [36]:
gpt3_ref_data = pd.DataFrame({
    'user_input': gpt3_inputs,
    'output_text': gpt3_response_texts,
    'token_likelihoods': gpt3_token_likelihoods_ref,
    'finish_reason': gpt3_response_finish_reasons_ref,
    'readability': gpt3_readability_ref,
    'correct': gpt3_correctness_ref,
    'user_feedback': gpt3_pseudofeedback_messages_ref,
    'user_feedback_score': gpt3_feedback_scores_ref
})
gpt3_ref_data

,user_input,output_text,token_likelihoods,finish_reason,readability,correct,user_feedback,user_feedback_score
0,What is the answer to this question? question:...,\n\nA heterocyclic compound.,"[{' ': 0.9719332740255017}, {' ': 0.8780232720...",stop,9.21,True,Thank you for your answer. It seems to make se...,8
1,What is the answer to this question? question:...,\nConcave mirrors.,"[{' ': 0.616506014836217}, {'Con': 0.711767098...",stop,35.61,False,Your answer is correct and concise. Good job! 8,8
2,What is the answer to this question? question:...,\n\nthe greater volume of water in the oceans ...,"[{' ': 0.6660635117503467}, {' ': 0.7959428676...",stop,68.77,False,Thank you for your answer! It is concise and a...,8
3,What is the answer to this question? question:...,Oxygen,"[{' Oxy': 0.5321888880134025}, {'gen': 1.0}]",stop,36.62,True,Your answer is correct and makes sense. 10,10
4,What is the answer to this question? question:...,\n\nAn electron.,"[{' ': 0.931050450120296}, {' ': 0.93818160497...",stop,77.91,True,Your answer is correct and makes sense. 10.,10
5,What is the answer to this question? question:...,\n\nSmooth muscle,"[{' ': 0.9447135208387549}, {' ': 0.9742459225...",stop,77.91,True,"Perfect answer, that makes complete sense. 10",10
6,What is the answer to this question? question:...,\n\nThe diploid zygote forms during fertilizat...,"[{' ': 0.9993525613788917}, {' ': 0.9666258613...",length,63.36,False,sperm cell from the male fuses with the egg ce...,6
7,What is the answer to this question? question:...,"\n\nSound waves travel through a medium, such ...","[{' ': 0.9754343109872657}, {' ': 0.9430283303...",length,85.69,False,solid objects. The waves move through the medi...,-1
8,What is the answer to this question? question:...,\nCalories,"[{' ': 0.5643018362391617}, {'Cal': 0.54332531...",stop,36.62,False,"Yes, that answer makes sense. Good job! 10",10
9,What is the answer to this question? question:...,Water,[{' Water': 0.7065745798560009}],stop,36.62,True,"Great, that's correct! 10",10


In [37]:
gpt3_inf_data = pd.DataFrame({
    'user_input': gpt3_inputs_inf,
    'output_text': gpt3_response_texts_inf,
    'token_likelihoods': gpt3_token_likelihoods_inf,
    'finish_reason': gpt3_response_finish_reasons_inf,
    'readability': gpt3_readability_inf,
    'correct': gpt3_correctness_inf,
    'user_feedback': gpt3_pseudofeedback_messages_inf,
    'user_feedback_score': gpt3_feedback_scores_inf
})
gpt3_inf_data

,user_input,output_text,token_likelihoods,finish_reason,readability,correct,user_feedback,user_feedback_score
0,What is the answer to this question? question:...,\n\nA continuous string of earthquakes typical...,"[{' ': 0.9999181494599383}, {' ': 0.9976895259...",length,14.97,False,The answer is relevant and directly answers th...,7
1,What is the answer to this question? question:...,\n\nCalcium ion is an example of a chemical me...,"[{' ': 0.8948000799648458}, {' ': 0.9667472181...",stop,36.96,False,Your answer is correct and makes sense. 10,10
2,What is the answer to this question? question:...,\n\nOceanic crust,"[{' ': 0.5902928850008413}, {' ': 0.8303103520...",stop,77.91,True,Your answer is correct and concise. Good job! 9,9
3,What is the answer to this question? question:...,\n\nVelocity.,"[{' ': 0.6796033023097143}, {' ': 0.9852181394...",stop,-132.59,True,"Thank you for your answer, that is correct. 10",10
4,What is the answer to this question? question:...,\n\nParasites infect the host's body.,"[{' ': 0.9107658135746581}, {' ': 0.8979725401...",stop,66.40,False,"Thank you, that answer is correct and makes se...",10
5,What is the answer to this question? question:...,\n\nFish have short esophaguses because they n...,"[{' ': 0.9994668844566175}, {' ': 0.9882386073...",length,68.77,False,food without relying heavily on chewing or bre...,3
6,What is the answer to this question? question:...,\n\nA cell.,"[{' ': 0.9946317849415638}, {' ': 0.9615109841...",stop,120.21,True,The answer provided is correct and makes sense. 8,8
7,What is the answer to this question? question:...,\n\nMany mammals control their body temperatur...,"[{' ': 0.9982622710024586}, {' ': 0.8082337400...",length,41.36,False,"Sorry, your answer seems incomplete as you did...",-1
8,What is the answer to this question? question:...,\n\nMultiple genes.,"[{' ': 0.47655687647477846}, {' ': 0.605107833...",stop,35.61,True,"Your answer is partially correct. It is ""multi...",7
9,What is the answer to this question? question:...,Antioxidants.,"[{' Ant': 0.8001724596376508}, {'ioxid': 0.999...",stop,-132.59,True,Your answer is correct and precise. Well done! 10,10


In [38]:
gpt3_ref_data.to_csv('reference_data.csv')
gpt3_inf_data.to_csv('inference_data.csv')

## Registering Model With Arthur

### Setting Up Connection
Supply your login to authenticate with the platform.

In [18]:
from arthurai import ArthurAI
# connect to Arthur
# UNCOMMENT the two lines below and enter your details
arthur = ArthurAI(
    url="https://app.arthur.ai",  # you can also pass this through the ARTHUR_ENDPOINT_URL environment variable
    login="<your login here>",  # you can also pass this through the ARTHUR_LOGIN environment variable
)

Please enter password for admin: ········


### Registering Model Type

In [40]:
from arthurai.common.constants import InputType, OutputType, ValueType, Stage

We'll instantiate an [`ArthurModel`](https://docs.arthur.ai/sdk/sdk_v3/apiref/arthurai.core.models.ArthurModel.html) with the `ArthurAI.model()` method, which constructs a new local `ArthurModel` object. Later we'll use `ArthurModel.save()` to register this model with the Arthur platform.

We give the model a user-friendly `display_name` and allow the unique `partner_model_id` field to be automatically generated, but you can supply a unique identifier if it helps you map your models in Arthur to your other MLOps systems.

The `InputType` of a model specifies the general type of data your model ingests. The `OutputType` of a model specifies the modeling task at hand.

### Building the model by specifying attributes

We use a helper function to register the model attributes for the input and output text the model will process

In [41]:
# register arthur model type
arthur_model_gpt3 = arthur.model(
    display_name="OpenAI_GPT3",
    input_type=InputType.NLP, 
    output_type=OutputType.TokenSequence)

# register attributes for token sequence model
arthur_model_gpt3.build_token_sequence_model(
    input_column="user_input", 
    output_text_column="output_text",
    output_likelihood_column="token_likelihoods"
)

# register additional non-input attributes
arthur_model_gpt3.add_attribute(
    name='finish_reason', 
    stage=Stage.NonInputData,
    value_type=ValueType.String,
    categorical=True,
    categories=['length', 'stop']
)

arthur_model_gpt3.add_attribute(
    name="readability",
    stage=Stage.NonInputData,
    value_type=ValueType.Float,
    min_range=0,
    max_range=100
)

arthur_model_gpt3.add_attribute(
    name="correct",
    stage=Stage.NonInputData,
    value_type=ValueType.Boolean,
    categorical=True,
    categories=[True,False]
)

arthur_model_gpt3.add_attribute(
    name="user_feedback",
    stage=Stage.NonInputData,
    value_type=ValueType.String,
    categorical=True,
    is_unique=True
)

arthur_model_gpt3.add_attribute(
    name="user_feedback_score",
    stage=Stage.NonInputData,
    value_type=ValueType.Integer,
    categorical=True,
    categories=[-1,0,1,2,3,4,5,6,7,8,9,10]
)

arthur_model_gpt3.review()

,name,stage,value_type,categorical,is_unique,categories,bins,range,monitor_for_bias
0,user_input,PIPELINE_INPUT,UNSTRUCTURED_TEXT,True,True,[],None,"[None, None]",False
1,token_likelihoods,PREDICTED_VALUE,TOKEN_LIKELIHOODS,False,False,[],None,"[None, None]",False
2,output_text,PREDICTED_VALUE,UNSTRUCTURED_TEXT,True,False,[],None,"[None, None]",False
3,finish_reason,NON_INPUT_DATA,STRING,True,False,"[{value: length}, {value: stop}]",None,"[None, None]",False
4,readability,NON_INPUT_DATA,FLOAT,False,False,[],None,"[0, 100]",False
5,correct,NON_INPUT_DATA,BOOLEAN,True,False,"[{value: True}, {value: False}]",None,"[None, None]",False
6,user_feedback,NON_INPUT_DATA,STRING,True,True,[],None,"[None, None]",False
7,user_feedback_score,NON_INPUT_DATA,INTEGER,True,False,"[{value: -1}, {value: 0}, {value: 1}, {value: ...",None,"[None, None]",False


### Saving the Model

Before saving, be sure to review your model to make sure everything is correct. We already saw the model schema returned by `ArthurModel.build()`, but we have since changed our attribute congiruations. Therefore we call `ArthurModel.review()` to see that our changed attributes look correct before saving to the platform. See the [onboarding walkthrough on the Arthur docs](https://docs.arthur.ai/user-guide/walkthroughs/model-onboarding/index.html#review-model) for tips on reviewing your model.

Note that while we capture the ranges of the attributes in this schema, they don’t need to be exact and won’t affect any performance calculations. They’re used as metadata to configure plots in the online Arthur dashboard, but never affect data drift or any other computations.

In [43]:
# review the model attribute properties in the model schema
arthur_model_gpt3.review()

,name,stage,value_type,categorical,is_unique,categories,bins,range,monitor_for_bias
0,user_input,PIPELINE_INPUT,UNSTRUCTURED_TEXT,True,True,[],None,"[None, None]",False
1,token_likelihoods,PREDICTED_VALUE,TOKEN_LIKELIHOODS,False,False,[],None,"[None, None]",False
2,output_text,PREDICTED_VALUE,UNSTRUCTURED_TEXT,True,False,[],None,"[None, None]",False
3,finish_reason,NON_INPUT_DATA,STRING,True,False,"[{value: length}, {value: stop}]",None,"[None, None]",False
4,readability,NON_INPUT_DATA,FLOAT,False,False,[],None,"[0, 100]",False
5,correct,NON_INPUT_DATA,BOOLEAN,True,False,"[{value: True}, {value: False}]",None,"[None, None]",False
6,user_feedback,NON_INPUT_DATA,STRING,True,True,[],None,"[None, None]",False
7,user_feedback_score,NON_INPUT_DATA,INTEGER,True,False,"[{value: -1}, {value: 0}, {value: 1}, {value: ...",None,"[None, None]",False


Now, we save the model. 

Note that this will be the first call to send data to the Arthur platform so far in this example - no information has been sent yet to the platform.

The method `ArthurModel.save()` sends an API request to Arthur to validate your model - if there are any problems with your model schema, this method will result in an error informing you how to correct your model's configuration. If no errors are found, the model will be saved to the platform.

In [45]:
# validate the model and save it onto the Arthur platform
arthur_model_gpt3.save()

17:55:43 - arthurai - We have registered the  model with Arthur and are getting it ready to accept inferences...
17:56:28 - arthurai - Model Creation Completed successfully, you can now send Data to Arthur.


'0a9be18f-870b-4944-aef0-080e7d91efd8'

In [46]:
arthur_model_gpt3.set_reference_data(data=gpt3_ref_data['user_input'])

17:56:28 - arthurai - Starting upload (0.005 MB in 1 files), depending on data size this may take a few minutes
17:56:28 - arthurai - Upload completed: /var/folders/8v/8v36mrp907z7lp5d4cd7yf4h0000gn/T/tmpogzzckn6/0a9be18f-870b-4944-aef0-080e7d91efd8-0.parquet


({'counts': {'success': 50, 'failure': 0, 'total': 50}, 'failures': [[]]},
 {'dataset_close_result': {'message': 'success'}})

<a id='sending_inferences_content'></a>

## Sending Inferences

#### Using `ArthurModel.send_inferences()`

We send our inference data to the platform along with some unique IDs. Timestamps will be auto-generated for these inferences by the `send_inferences` function from the Arthur SDK. See [our API docs for sending inferences](https://docs.arthur.ai/api-documentation/v3-api-docs.html#tag/inferences/paths/~1models~1%7Bmodel_id%7D~1inferences/post) for the full specification of inference ingestion in Arthur.

In [48]:
inference_result = arthur_model_gpt3.send_inferences(
    gpt3_inf_data, 
    partner_inference_ids=[shortuuid.uuid() for _ in range(len(gpt3_inputs_inf))])
print('# of successful and # of failing inference uploads:', inference_result['counts'])

18:06:20 - arthurai - 50 rows were missing inference_timestamp fields, so the current time was populated
# of successful and # of failing inference uploads: {'failure': 0, 'success': 50, 'total': 50}


## See Model in Dashboard

In [49]:
# the code below will render a link for you to view your model in the Arthur Dashboard

def render_arthur_model_dashboard_link(arthur, arthur_model):
    url = 'https://' + ''.join(arthur.client.api_base_url.split('/')[1:-2])
    link_text = f"See your model ({arthur_model.display_name}) in the Arthur Dashboard"
    href_string = f"{url}/model/{arthur_model.id}/overview"
    html_string = f'<br> <a style="font-size:200%" href={href_string}>{link_text}</a> <br>'
    display(HTML(html_string))

render_arthur_model_dashboard_link(arthur, arthur_model_gpt3) 

Once your inference data has been uploaded to the platform, you can see your model by following the above link to the model dashboard page to see an overview of the model and browse its inference data.

---